# Link Prediction

## 1. install the necessary packages

In [1]:
!pip install networkx numpy torch torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.0 MB/s eta 0:00:00


In [2]:
!pip install node2vec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.2 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.2.1
    Uninstalling networkx-3.2.1:
      Successfully uninstalled networkx-3.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


## 2. Read the dataset generated from knowledge graph

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Upload the knowledge graph data with 2 csv files
import pandas as pd
nodes_company = pd.read_csv("/content/drive/MyDrive/hidy.nodes.company.csv")
KG_GNNinput = pd.read_csv("/content/drive/MyDrive/knowledge_graph_GNNinput.csv")

In [5]:
# Construct a networkx graph data structure
import networkx as nx

G = nx.Graph()

for i, row in nodes_company.iterrows():
    node_id = row[":ID"]
    G.add_node(node_id)

for i, row in KG_GNNinput.iterrows():
    start_node = row["start_node_id"]
    end_node = row["end_node_id"]
    G.add_edge(start_node, end_node, label = row["label"])

In [6]:
# Show the detail of the knowledge graph
print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())

print("List of nodes:", list(G.nodes()))
print("List of edges:", list(G.edges()))

Number of nodes: 3974
Number of edges: 8998
List of nodes: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 

## 3. Get the nodes' feature matrix and build the adjancency matrix

In [7]:
from gensim.models import Word2Vec

# Use Word2Vec method to get the nodes' feature vector
sentences = [list(G.neighbors(node)) for node in G.nodes()]
model = Word2Vec(sentences, vector_size=64, window=5, min_count=1, sg=1, workers=4)
node_embeddings = model.wv
node_features = node_embeddings.vectors

print("Node feature matrix:\n", node_features)

Node feature matrix:
 [[ 6.79165304e-01 -4.87731546e-01  3.25046957e-01 ... -8.05401206e-02
   8.96630436e-02  1.08323231e-01]
 [ 7.13948429e-01 -5.12078464e-01  3.36153686e-01 ... -8.08716118e-02
   9.46583599e-02  1.37973458e-01]
 [ 7.10235417e-01 -5.00028908e-01  3.48119795e-01 ... -9.56268907e-02
   9.98819619e-02  1.26227558e-01]
 ...
 [ 1.59754790e-02 -2.01674290e-02 -9.10245068e-03 ...  1.21766492e-03
   1.01399105e-02  9.49612167e-03]
 [ 4.03808728e-02 -2.59525236e-02  1.74712017e-02 ... -3.19069740e-03
  -7.22822826e-03 -4.39268444e-03]
 [ 2.89523900e-02  2.18901550e-05  1.00768525e-02 ... -1.22115184e-02
   2.28025229e-03 -8.17623828e-03]]


In [8]:
def build_adjacency_matrix(graph):
    adjacency_matrix = nx.adjacency_matrix(graph)
    return adjacency_matrix

In [9]:
import pandas as pd
import numpy as np
import networkx as nx
import torch
from sklearn.model_selection import train_test_split
from torch_geometric.data import Data

# Get all nodes and edges
all_nodes = list(G.nodes())
all_edges = list(G.edges())
all_labels = [[G[edge[0]][edge[1]]['label']] for edge in all_edges]

## 4. Create the no-link data with same number of validation and test datasets and split all data

In [10]:
# Create no-link data for validation and test sets
non_edges = []
while len(non_edges) < len(all_edges):
    node1, node2 = np.random.choice(all_nodes, 2)
    if (node1, node2) not in all_edges and (node2, node1) not in all_edges:
        non_edges.append((node1, node2))
        # G.add_edge(node1, node2, label=-1)

In [11]:
# Split the datasets with train, validation and test sets
X = all_edges + non_edges
y = all_labels + [[-1]] * len(non_edges)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [12]:
# Transform the data to the type which PyTorch Geometric accepts

train_data = Data(edge_index=torch.tensor(X_train, dtype=torch.long).t(), x=torch.tensor(y_train, dtype=torch.float))
val_data = Data(edge_index=torch.tensor(X_val, dtype=torch.long).t(), x=torch.tensor(y_val, dtype=torch.float))
test_data = Data(edge_index=torch.tensor(X_test, dtype=torch.long).t(), x=torch.tensor(y_test, dtype=torch.float))

In [13]:
print(train_data.edge_index)

tensor([[1220, 3796, 1385,  ..., 2297, 2558, 1245],
        [1632, 2301,  823,  ..., 3711, 3030,  533]])


In [14]:
print(train_data)
print(test_data)
print(val_data)

Data(x=[10797, 1], edge_index=[2, 10797])
Data(x=[3600, 1], edge_index=[2, 3600])
Data(x=[3599, 1], edge_index=[2, 3599])


In [15]:
print(train_data.num_features)

1


## 5. GNN Model

In [16]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(train_data.num_features, 128)
        self.conv2 = GCNConv(128, 64)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return torch.sigmoid(x)

In [ ]:
model = Net()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Train the model
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(train_data)
    loss = F.binary_cross_entropy(out[train_data.edge_index], train_data.x)
    # loss = F.binary_cross_entropy(out[train_data.edge_index], train_data.y)
    loss.backward()
    optimizer.step()

In [ ]:
# Test the model
model.eval()
with torch.no_grad():
    pred = model(test_data).detach().numpy()

# Turn the prediction values to 0,1,-1
pred = np.where(pred > 0.5, 1, 0)
pred = np.where(pred < 0.5, 0, pred)
pred = np.where(test_data.x == -1, -1, pred)